In [14]:
import pandas as pd
import numpy as np
from libreco.data import DatasetFeat, split_multi_value
from libreco.algorithms import DeepFM
import tensorflow.compat.v1 as tf

# Metricas

In [15]:
animes = pd.read_csv('clean_data/animes.csv')
profiles = pd.read_csv('clean_data/profiles.csv')

In [16]:
animes

,uid,title,synopsis,genre,aired,episodes,members,popularity,ranked,score,img_url,link,release_year,is_ongoing
0,28891,Haikyuu!! Second Season,Following their participation at the Inter-Hig...,"['Comedy', 'Sports', 'Drama', 'School', 'Shoun...","Oct 4, 2015 to Mar 27, 2016",25.0,489888,141,25.0,8.82,https://cdn.myanimelist.net/images/anime/9/766...,https://myanimelist.net/anime/28891/Haikyuu_Se...,2015.0,0
1,23273,Shigatsu wa Kimi no Uso,Music accompanies the path of the human metron...,"['Drama', 'Music', 'Romance', 'School', 'Shoun...","Oct 10, 2014 to Mar 20, 2015",22.0,995473,28,24.0,8.83,https://cdn.myanimelist.net/images/anime/3/671...,https://myanimelist.net/anime/23273/Shigatsu_w...,2014.0,0
2,34599,Made in Abyss,The Abyss—a gaping chasm stretching down into ...,"['Sci-Fi', 'Adventure', 'Mystery', 'Drama', 'F...","Jul 7, 2017 to Sep 29, 2017",13.0,581663,98,23.0,8.83,https://cdn.myanimelist.net/images/anime/6/867...,https://myanimelist.net/anime/34599/Made_in_Abyss,2017.0,0
3,5114,Fullmetal Alchemist: Brotherhood,"""In order for something to be obtained, someth...","['Action', 'Military', 'Adventure', 'Comedy', ...","Apr 5, 2009 to Jul 4, 2010",64.0,1615084,4,1.0,9.23,https://cdn.myanimelist.net/images/anime/1223/...,https://myanimelist.net/anime/5114/Fullmetal_A...,2009.0,0
4,31758,Kizumonogatari III: Reiketsu-hen,After helping revive the legendary vampire Kis...,"['Action', 'Mystery', 'Supernatural', 'Vampire']","Jan 6, 2017",1.0,214621,502,22.0,8.83,https://cdn.myanimelist.net/images/anime/3/815...,https://myanimelist.net/anime/31758/Kizumonoga...,2017.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19306,32979,Flip Flappers,Cocona is an average middle schooler living wi...,"['Sci-Fi', 'Adventure', 'Comedy', 'Magic']","Oct 6, 2016 to Dec 29, 2016",13.0,134252,843,1070.0,7.73,https://cdn.myanimelist.net/images/anime/4/822...,https://myanimelist.net/anime/32979/Flip_Flappers,2016.0,0
19307,123,Fushigi Yuugi,"While visiting the National Library, junior-hi...","['Adventure', 'Fantasy', 'Magic', 'Martial Art...","Apr 6, 1995 to Mar 28, 1996",52.0,84407,1292,1071.0,7.73,https://cdn.myanimelist.net/images/anime/2/201...,https://myanimelist.net/anime/123/Fushigi_Yuugi,1995.0,0
19308,1281,Gakkou no Kaidan,"Years ago, all of the ghosts in a haunted scho...","['Mystery', 'Horror', 'Supernatural']","Oct 22, 2000 to Mar 25, 2001",19.0,83093,1314,1073.0,7.73,https://cdn.myanimelist.net/images/anime/9/183...,https://myanimelist.net/anime/1281/Gakkou_no_K...,2000.0,0
19309,450,InuYasha Movie 2: Kagami no Naka no Mugenjo,Inuyasha and company have finally destroyed Na...,"['Action', 'Adventure', 'Comedy', 'Historical'...","Dec 21, 2002",1.0,71989,1469,1077.0,7.73,https://cdn.myanimelist.net/images/anime/1162/...,https://myanimelist.net/anime/450/InuYasha_Mov...,2002.0,0


In [17]:
profiles

,profile,gender,birthday,favorites_anime,link,user_id,birthday_year,edad
0,DesolatePsyche,Male,"Oct 2, 1994","['33352', '25013', '5530', '33674', '1482', '2...",https://myanimelist.net/profile/DesolatePsyche,7.0,1994.0,31.0
1,baekbeans,Female,"Nov 10, 2000","['11061', '31964', '853', '20583', '918', '925...",https://myanimelist.net/profile/baekbeans,0.0,2000.0,25.0
2,skrn,NaN,NaN,"['918', '2904', '11741', '17074', '23273', '32...",https://myanimelist.net/profile/skrn,1.0,NaN,NaN
3,aManOfCulture99,Male,"Oct 30, 1999","['4181', '7791', '9617', '5680', '2167', '4382...",https://myanimelist.net/profile/aManOfCulture99,3.0,1999.0,26.0
4,eneri,NaN,NaN,"['5114', '4898', '2904', '1575', '1482']",https://myanimelist.net/profile/eneri,4.0,NaN,NaN
...,...,...,...,...,...,...,...,...
74172,samdineen20,Female,"Apr 18, 1992","['120', '150', '3457', '3655', '4898']",https://myanimelist.net/profile/samdineen20,24117.0,1992.0,33.0
74173,anjel,Female,"Feb 28, 1988",[],https://myanimelist.net/profile/anjel,24118.0,1988.0,37.0
74174,lovelessxd,Female,"Aug 6, 1992","['853', '5114']",https://myanimelist.net/profile/lovelessxd,21856.0,1992.0,33.0
74175,Shattered_Angel,Female,"Sep 6, 1994","['150', '27', '1520', '121', '31452', '32995',...",https://myanimelist.net/profile/Shattered_Angel,10783.0,1994.0,31.0


In [18]:
df_train = pd.read_csv(
    "train", sep=",", names=["userid", "itemid", "rating"], header=None
)

df_train.head()

,userid,itemid,rating
0,1635,6675.0,9
1,3340,7785.0,10
2,1015,578.0,6
3,87,18617.0,9
4,5582,8425.0,8


In [19]:
df_test = pd.read_csv(
    "test", sep=",", names=["userid", "itemid", "rating"], header=None
)

df_test.head()

,userid,itemid,rating
0,20408,8426.0,8
1,22049,5114.0,10
2,4728,4224.0,10
3,690,6746.0,9
4,254,481.0,6


In [20]:
def add_extra_features(df, profiles, animes):
    # Join with profiles to get user features
    df = df.merge(profiles, how='left', left_on='userid', right_on='user_id')

    # Join with animes to get item features
    df = df.merge(animes, how='left', left_on='itemid', right_on='uid')
    
    # Drop unnecessary columns
    df = df.drop(columns=['user_id', 'uid', 'profile', 'birthday', 'favorites_anime', 'link_x', 'birthday_year'])
    
    return df

def separate_genre_column(df):
    # genre column is a string representing a list of genres (e.g. "['Action', 'Adventure']")
    df['genre_list'] = df['genre'].apply(lambda x: eval(x) if pd.notnull(x) else [])
    # Create separate columns for up to 3 genres
    for i in range(3):
        df[f'genre_{i+1}'] = df['genre_list'].apply(lambda x: x[i] if len(x) > i else None)
        
    df = df.drop(columns=['genre', 'genre_list'])
    return df

def rename_columns(df):
    df = df.rename(columns={
        'itemid': 'item', 'userid': 'user', 'rating': 'label'
    })
    return df

In [21]:
df_train = add_extra_features(df_train, profiles, animes)
df_test = add_extra_features(df_test, profiles, animes)

df_train = separate_genre_column(df_train)
df_test = separate_genre_column(df_test)

df_train = rename_columns(df_train)
df_test = rename_columns(df_test)

In [22]:
df_train.head(3)

,user,item,label,gender,edad,title,synopsis,aired,episodes,members,popularity,ranked,score,img_url,link_y,release_year,is_ongoing,genre_1,genre_2,genre_3
0,1635,6675.0,9,Male,32.0,Redline,"Every five years, an exhilarating race called ...","Aug 14, 2009",1.0,207573,523,227.0,8.31,https://cdn.myanimelist.net/images/anime/12/28...,https://myanimelist.net/anime/6675/Redline,2009.0,0,Action,Sci-Fi,Cars
1,1635,6675.0,9,Male,32.0,Redline,"Every five years, an exhilarating race called ...","Aug 14, 2009",1.0,207573,523,227.0,8.31,https://cdn.myanimelist.net/images/anime/12/28...,https://myanimelist.net/anime/6675/Redline,2009.0,0,Action,Sci-Fi,Cars
2,1635,6675.0,9,Male,32.0,Redline,"Every five years, an exhilarating race called ...","Aug 14, 2009",1.0,207573,523,227.0,8.31,https://cdn.myanimelist.net/images/anime/12/28...,https://myanimelist.net/anime/6675/Redline,2009.0,0,Action,Sci-Fi,Cars


In [23]:
sparse_col = ["gender", "is_ongoing", "genre_1", "genre_2", "genre_3"]
dense_col = ["edad", "members", "popularity", "episodes", "score", "release_year"]

user_col = ["gender", "edad"]
item_col = [
    "genre_1", "genre_2", "genre_3", "release_year", "members", "popularity", "episodes", "score", "is_ongoing"
]

# Fix missing values safely
for col in sparse_col:
    df_train[col] = df_train[col].fillna("unknown").astype(str)

for col in dense_col:
    # 1. Calculate the median of the existing values
    median_val = df_train[col].median()
    
    # 2. Fill NaNs with that median
    df_train[col] = df_train[col].fillna(median_val).astype(float)

    print(f"Column '{col}' missing values filled with median: {median_val}")

from sklearn.preprocessing import MinMaxScaler

# --- 1. Log Transform Skewed Features ---
# We use log1p (log(x+1)) to avoid errors with 0 values
# This compresses 1,000,000 down to roughly 13.8, making it manageable
for col in ["members", "popularity"]: # Add whatever high-number cols you have
    if col in df_train.columns:
        df_train[col] = np.log1p(df_train[col].fillna(0))

# --- 2. Min-Max Scale All Dense Features ---
# This forces every dense number to be between 0 and 1
scaler = MinMaxScaler()
df_train[dense_col] = scaler.fit_transform(df_train[dense_col])

# Example checks (Run these to verify!)
print("Max Members (Scaled):", df_train['members'].max()) # Should be 1.0
print("Max Year (Scaled):", df_train['release_year'].max()) # Should be 1.0

train_data, data_info = DatasetFeat.build_trainset(
    df_train, 
    user_col, 
    item_col, 
    sparse_col, 
    dense_col,
)
test_data = DatasetFeat.build_testset(df_test)

Column 'edad' missing values filled with median: 31.0
Column 'members' missing values filled with median: 320630.0
Column 'popularity' missing values filled with median: 301.0
Column 'episodes' missing values filled with median: 13.0
Column 'score' missing values filled with median: 8.06
Column 'release_year' missing values filled with median: 2011.0
Max Members (Scaled): 1.0
Max Year (Scaled): 1.0


In [24]:
tf.reset_default_graph()

model = DeepFM(
    task="ranking",
    data_info=data_info,
    embed_size=32,          # Increased from 16
    n_epochs=15,            # Increased from 3
    lr=1e-3,                # Increased slightly, watch for divergence
    batch_size=256,         # Smaller batch size often generalizes better
    use_bn=True,
    dropout_rate=0.2,       # Added dropout to prevent overfitting
    hidden_units=(128, 64, 32),
)

model.fit(
    train_data,
    neg_sampling=True,
    verbose=2,
    shuffle=True,
    eval_data=test_data,
    metrics=["loss", "roc_auc", "precision", "ndcg"], # Add ranking metrics
)

Training start time: 2025-11-29 16:48:10
Instructions for updating:
Colocations handled automatically by placer.


c:\Users\benja\Documents\Universidad\2025-2\Recomendadores\Proyecto\.venv\lib\site-packages\libreco\layers\dense.py:31: UserWarning: `tf.layers.batch_normalization` is deprecated and will be removed in a future version. Please use `tf.keras.layers.BatchNormalization` instead. In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.BatchNormalization` documentation).
  net = tf.layers.batch_normalization(net, training=is_training)
2025-11-29 16:48:10,856 - WARNING - From c:\Users\benja\Documents\Universidad\2025-2\Recomendadores\Proyecto\.venv\lib\site-packages\keras\layers\normalization\batch_normalization.py:581: _colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


total params: 1,236,475 | embedding params: 1,171,693 | network params: 64,782

c:\Users\benja\Documents\Universidad\2025-2\Recomendadores\Proyecto\.venv\lib\site-packages\libreco\layers\dense.py:39: UserWarning: `tf.layers.batch_normalization` is deprecated and will be removed in a future version. Please use `tf.keras.layers.BatchNormalization` instead. In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.BatchNormalization` documentation).
  net = tf.layers.batch_normalization(net, training=is_training)
c:\Users\benja\Documents\Universidad\2025-2\Recomendadores\Proyecto\.venv\lib\site-packages\libreco\layers\dense.py:47: UserWarning: `tf.layers.dropout` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dropout` instead.
  net = tf.layers.dropout(net, dropout_rate, training=is_training)


train: 100%|██████████| 4527/4527 [00:21<00:00, 211.62it/s]


...

Detect 43 unknown interaction(s), position: [3840, 3842, 5636, 5638, 3346, 4370, 3348, 4372, 3350, 3352, 1958, 5798, 1960, 2728, 1962, 4008, 1964, 4010, 1966, 4012, 5800, 4030, 4032, 4034, 4036, 3398, 3400, 5704, 3402, 5706, 3404, 5074, 3156, 5076, 5078, 5866, 5868, 3438, 5870, 3440, 5872, 3836, 3838]
Detect 34 unknown interaction(s), position: [4224, 1410, 1412, 42, 3754, 3244, 3756, 3246, 3758, 3248, 5802, 3250, 3252, 3254, 6592, 1608, 3704, 976, 978, 3706, 6614, 6616, 3708, 5344, 1634, 1636, 1638, 4222, 2420, 2422, 2424, 2426, 2428, 3710]
Detect 56 unknown interaction(s), position: [256, 258, 4580, 5398, 6434, 6436, 6192, 6194, 5428, 6196, 5430, 6198, 5432, 6200, 2746, 5434, 700, 5820, 702, 5822, 704, 5824, 706, 7870, 7872, 3450, 4690, 3166, 4574, 352, 3168, 354, 3170, 356, 3172, 358, 3174, 360, 3176, 362, 4320, 4324, 4326, 4578, 4582, 4576, 4322, 5618, 1012, 5620, 1014, 5622, 1016, 1018, 252, 254]
Detect 42 unknown interaction(s), position: [1536, 6144, 6146, 7424, 6148, 6150, 7052

eval_pointwise: 100%|██████████| 35/35 [00:00<00:00, 60.14it/s]

Detect 35 unknown interaction(s), position: [6528, 7936, 7938, 7940, 6664, 3466, 6666, 3468, 6668, 6670, 1684, 3616, 7584, 3618, 7586, 3620, 7588, 3622, 7590, 8122, 8124, 8126, 8128, 3044, 7932, 3046, 3048, 3050, 3052, 7934, 4594, 4596, 7930, 6524, 6526]
Detect 16 unknown interaction(s), position: [3526, 6872, 4824, 6874, 4826, 2404, 4068, 2406, 4070, 2408, 4072, 2410, 6694, 2412, 7980, 6320]
Detect 34 unknown interaction(s), position: [4882, 4884, 4886, 1690, 1564, 1566, 1568, 1570, 1572, 1574, 1576, 1578, 5162, 1580, 5164, 1582, 2990, 432, 2992, 434, 5166, 5168, 1464, 1466, 1468, 1470, 1472, 4952, 4954, 4956, 4958, 4960, 8054, 8056]
Detect 25 unknown interaction(s), position: [2824, 5624, 2826, 5770, 1298, 3858, 1300, 3860, 3862, 5626, 2712, 7960, 7962, 7964, 7966, 6384, 50, 5618, 2356, 5620, 1078, 2358, 2360, 5622, 2362]
Detect 47 unknown interaction(s), position: [1668, 1670, 1672, 1674, 4490, 1676, 4492, 1678, 4494, 4496, 8074, 8076, 6676, 6678, 6680, 6682, 6684, 2856, 5800, 2858,


eval_listwise: 100%|██████████| 15088/15088 [03:10<00:00, 79.04it/s]


	 eval log_loss: 0.7175
	 eval roc_auc: 0.9646
	 eval precision@10: 0.0020
	 eval ndcg@10: 0.0085


In [25]:
# predict preference of user 2211 to item 110
print(model.predict(user=506, item=110)
)# recommend 7 items for user 2211
print(model.recommend_user(user=506, n_rec=10))
print(model.recommend_user(user=506, n_rec=10)[506])

3.8576978e-08
{506: array([3.4599e+04, 4.1770e+03, 3.7999e+04, 1.9000e+01, 1.8180e+03,
       3.8691e+04, 3.5790e+04, 2.1843e+04, 3.6296e+04, 2.0200e+02])}
[3.4599e+04 4.1770e+03 3.7999e+04 1.9000e+01 1.8180e+03 3.8691e+04
 3.5790e+04 2.1843e+04 3.6296e+04 2.0200e+02]


In [26]:
# cold-start prediction
print(model.predict(user="ccc", item="not item", cold_start="average"))
# cold-start recommendation
print(model.recommend_user(user="are we good?", n_rec=7, cold_start="popular"))

Detect 1 unknown interaction(s), position: [0]
0.9637716
Detect unknown user: are we good?
{'are we good?': array([ 4224.,  4898.,  3588., 13125., 13125., 12355.,  4224.])}


In [27]:
from evaluate import get_metrics

def get_recommendations(user_id, n_rec=10):
    return model.recommend_user(user_id, n_rec=n_rec)[user_id]

def undo_rename_columns(df):
    df = df.rename(columns={
        'item': 'itemid', 'user': 'userid', 'label': 'rating'
    })
    return df

df_train = undo_rename_columns(df_train)
df_test = undo_rename_columns(df_test)

get_metrics(df_train, df_test, get_recommendations, k=10)

Evaluando usuarios: 100%|██████████| 15089/15089 [01:58<00:00, 127.79it/s]

--- Métricas Globales de Evaluación ---
{
  "mean_recall": 0.021878155503197577,
  "mean_precision": 0.0022775720857174915,
  "mean_ap (MAP)": 0.006168669270318547,
  "mean_ndcg": 0.009794890408601854,
  "mean_novelty": 5.078787323111179,
  "mean_diversity": 0.7935558721135033,
  "num_users_evaluated": 8913
}

--- Reporte de Fairness (Disparidad de Grupo) ---
{
  "delta_threshold": 0.2,
  "group_averages": {
    "Female": {
      "recall": 0.019246519246519246,
      "precision": 0.0019246519246519248,
      "MAP": 0.005869180869180869,
      "nDCG": 0.008962826482510906,
      "novelty": 5.21634565568845,
      "diversity": 0.7908070567919053,
      "count": 2442
    },
    "Male": {
      "recall": 0.02287127182815639,
      "precision": 0.002410755679184052,
      "MAP": 0.006281689000743243,
      "nDCG": 0.010108891352430332,
      "novelty": 5.026876111837235,
      "diversity": 0.7945932090035269,
      "count": 6471
    }
  },
  "disparity_reports": [
    {
      "pair": [
    

In [1]:
get_metrics(df_train, df_test, get_recommendations, k=10, use_age_group=True)

NameError: name 'get_metrics' is not defined